In [12]:
import os
os.getcwd()

'C:\\Users\\maria\\Documents\\DATA PROJECTS\\OnlineRetailProject\\notebooks'

In [13]:
import pandas as pd

# Load the dataset
file_path = 'C:\\Users\\maria\\Documents\\DATA PROJECTS\\OnlineRetailProject\\data\\raw\\OnlineRetail.csv'

# Loading the CSV file using ISO-8859-1 encoding instead of UTF-8.
# This avoids UnicodeDecodeError caused by special characters (e.g., £, accented letters).
df = pd.read_csv(file_path, encoding='ISO-8859-1')

# Preview
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [14]:
# INITIAL EXPLORATION
# Check the structure and data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [15]:
# Check for null values
print("\nNull values per column:")
print(df.isnull().sum())


Null values per column:
InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64


In [16]:
# Check for duplicates
print("\nNumber of duplicate rows:")
print(df.duplicated().sum())


Number of duplicate rows:
5268


In [17]:
# Check for Quantity issues (e.g. zero or negative)
print("\nNumber of rows with Quantity <= 0:")
print((df['Quantity'] <= 0).sum())


Number of rows with Quantity <= 0:
10624


In [18]:
# Check for returns ((cancelled orders have InvoiceNo starting with 'C')
print("\nNumber of Returns")
print((df['InvoiceNo'].astype(str).str.startswith('C')).sum())


Number of Returns
9288


In [19]:
# CLEANING STEPS

# Drop rows missing key information (CustomerID or Description)
df.dropna(subset=['CustomerID', 'Description'], inplace=True)

# Remove transactions with zero or negative Quantity
df = df[df['Quantity'] != 0]

# Add a flag for returns (cancelled orders have InvoiceNo starting with 'C')
df['IsReturn'] = df['InvoiceNo'].astype(str).str.startswith('C').astype(int)

# Convert InvoiceDate to datetime for time-based analysis
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

# Drop any duplicate rows
df.drop_duplicates(inplace=True)

# Create TotalPrice column (line item total)
df['TotalPrice'] = df['Quantity'] * df['UnitPrice']

# Add Year and Month columns for trend analysis
df['Year'] = df['InvoiceDate'].dt.year
df['Month'] = df['InvoiceDate'].dt.month


In [20]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,IsReturn,TotalPrice,Year,Month
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,0,15.30,2010,12
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,0,20.34,2010,12
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,0,22.00,2010,12
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,0,20.34,2010,12
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,0,20.34,2010,12


In [21]:
df['IsReturn'].value_counts()

IsReturn
0    392732
1      8872
Name: count, dtype: int64

In [23]:
# Export cleaned dataset
export_path = 'C:/ProgramData/MySQL/MySQL Server 8.0/Uploads/CleanedOnlineRetail.csv'
df.to_csv(export_path, index=False, encoding='ISO-8859-1')


In [ ]:
df.to_csv("CleanedOnlineRetail.csv", index=False, encoding="ISO-8859-1")